https://www.analyticsvidhya.com/blog/2021/06/k-means-clustering-and-transfer-learning-for-image-classification/

https://medium.com/@joel_34096/k-means-clustering-for-image-classification-a648f28bdc47

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
from scipy.stats import mode

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.cluster import  KMeans

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import pandas as pd

from time import time
import os
import pathlib
import shutil
from skimage import io
import cv2

In [ ]:
# self-written scripts
import sys
sys.path.insert(0, 'Python_Scripts')

import util

In [ ]:
# Loading the complete dataset (including class 0)
df = pd.read_csv('data/train_complete.csv')
df.head()

In [ ]:
# Loading the train dataset without class 0
df2 = pd.read_csv('data/train.csv')
df2.head()

In [ ]:
util.isolate_single_defects(df2)

In [ ]:
df2.sample(10)

In [ ]:
# isolate only images that have 0 or 1 defect
util.isolate_single_defects(df)

In [ ]:
# eliminate class 0 images in dataset
df = df.query('ClassId!=0')
df.shape

In [ ]:
# safe all images with single defects
path = pathlib.Path.cwd()
try:
    os.mkdir(path.joinpath('data','single_defect_train_images'))
except:
    print('Folder already exists.')
    # von Michael kopiert für Ordnererstellung basierend auf x_train
    for i in range(len(df)):
        origin_train_path = path.joinpath('data', 'train_images')
        source_file = df.iloc[i,1]
        target_directory = path.joinpath('data', 'single_defect_train_images')
        shutil.copy2(origin_train_path.joinpath(source_file) , target_directory.joinpath(source_file))


In [ ]:
df.head()

### Preprozessing for single image (Example)

In [ ]:
# load image (example)
images = io.imread('data/single_defect_train_images/0002cc93b.jpg')
print('Org image shape --> ',images.shape)

In [ ]:
plt.rcParams['axes.grid'] = False
plt.imshow(images)

In [ ]:
#images[:,0]

In [ ]:
print(images.max())

In [ ]:
#resize images
IMG_SIZE = 32
image_resized=cv2.resize(images,(IMG_SIZE,IMG_SIZE))
print('After resizing shape --> ',image_resized.shape)

In [ ]:
plt.rcParams['axes.grid'] = False
plt.imshow(image_resized)

In [ ]:
print(len(image_resized))

In [ ]:
print(image_resized.max())

In [ ]:
#The input data have to be converted from 3 dimensional format to 1 dimensional format to be fed into the K-Means Clustering algorithm (Reshape images)
#image_flat = image_resized.reshape(len(image_resized),-1)
image_flat = image_resized.reshape(1, 3*IMG_SIZE*IMG_SIZE)

print('After Flattening shape --> ',image_flat.shape)

In [ ]:
image_flat

In [ ]:
print(image_flat.max())

In [ ]:
flattened_images =[]

IMG_SIZE = 128
for image_id in df['ImageId']:
    image = io.imread('data/single_defect_train_images/' + image_id)
    #resize images
    image_resized=cv2.resize(image,(IMG_SIZE,IMG_SIZE))
    #The input data have to be converted from 3 dimensional format to 1 dimensional format
    image_flat = image_resized.reshape(1, 3*IMG_SIZE*IMG_SIZE)
    # Data Normalization
    # Conversion to float
    image_flat=image_flat.astype('float32')
    # Normalization (In the RGB color space the red, green and blue have integer values from 0 to 255)
    image_flat = image_flat/255.0
    flattened_images.append([image_id,image_flat[0]])

In [ ]:
flattened_images= pd.DataFrame(flattened_images , columns = ['ImageId', 'flattened_images'])
flattened_images

In [ ]:
flattened_images.shape

In [ ]:
split_flattened_images = pd.DataFrame(flattened_images['flattened_images'].tolist())
#split_flattened_images

In [ ]:
flattened_images = pd.concat([flattened_images, split_flattened_images], axis=1)
#flattened_images

In [ ]:
flattened_images_X=flattened_images.drop(['ImageId','flattened_images'], axis = 1)
#flattened_images_X

In [ ]:
# Use TSNE before modeling
tsne = TSNE(n_components=2, init='random', random_state=0)
digits_proj = tsne.fit_transform(flattened_images_X)
# Creating the KMeans model and predict classes (n_clusters =4 ; due to 4 classes)
kmeans = KMeans(n_clusters=4, random_state=0)
clusters = kmeans.fit_predict(digits_proj)
kmeans.cluster_centers_.shape

In [ ]:
#without TSNE
#Creating the KMeans model and predict classes (n_clusters =4 ; due to 4 classes)
#kmeans = KMeans(n_clusters=4, random_state=0)
#clusters = kmeans.fit_predict(flattened_images_X)
#kmeans.cluster_centers_.shape

In [ ]:
clusters

In [ ]:
clusters= pd.DataFrame(clusters,columns = ['ClassId_pred'])

In [ ]:
clusters

In [ ]:
df

In [ ]:
df.reset_index(drop=True, inplace=True)
clusters.reset_index(drop=True, inplace=True)
result = pd.concat([df, clusters], axis=1)
result['ClassId_pred'] = result['ClassId_pred'].map({0: 1, 1: 2,2:3, 3:4})
#switch labels due to results from confusion matrix
#result['ClassId_pred'] = result['ClassId_pred'].map({1: 1, 2: 2, 3:4, 4:3})
result

In [ ]:
result.groupby('ClassId').count()

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(result['ClassId'],result['ClassId_pred']))
# Print confusion matrix
sns.heatmap(metrics.confusion_matrix(result['ClassId'],result['ClassId_pred']), annot=True, cmap='YlGn');

769
195
4759
516


In [ ]:
print(metrics.classification_report(result['ClassId'],result['ClassId_pred']))

 ####  without TSNE and img_size = 128
   
           precision    recall  f1-score   support

           1       0.06      0.16      0.09       769
           2       0.04      0.18      0.06       195
           3       0.71      0.39      0.51      4759
           4       0.07      0.11      0.09       516

    accuracy                           0.34      6239
   macro avg       0.22      0.21      0.19      6239
weighted avg       0.56      0.34      0.41      6239

 #### with TSNE  and img_size = 128  
            precision    recall  f1-score   support

           1       0.18      0.40      0.25       769
           2       0.09      0.38      0.14       195
           3       0.68      0.28      0.40      4759
           4       0.11      0.37      0.17       516

    accuracy                           0.31      6239
   macro avg       0.26      0.36      0.24      6239
weighted avg       0.55      0.31      0.35      6239

## Clustering with HOG Features

In [ ]:
# Loading the HOG feature dataset 
hog = pd.read_csv('data/train_HOG.csv')
hog.head()

In [ ]:
hog.shape

In [ ]:
# isolate only images that have 0 or 1 defect
util.isolate_single_defects(hog)

In [ ]:
hog.shape

In [ ]:
# Loading the complete dataset (including class 0)
df2 = pd.read_csv('data/train_complete.csv')
df2.head()

In [ ]:
hog = hog.merge(df2, on='ImageId')
hog

In [ ]:
# eliminate class 0 images in dataset
hog = hog.query('ClassId!=0')
hog.shape

In [ ]:
hog_X=hog.drop(['ImageId','FilePath','ClassId','EncodedPixels','Defect'], axis = 1)

In [ ]:
'''# Use TSNE before modeling
tsne = TSNE(n_components=2, init='random', random_state=0)
digits_proj = tsne.fit_transform(hog_X)
# Creating the KMeans model and predict classes (n_clusters =4 ; due to 4 classes)
kmeans = KMeans(n_clusters=4, random_state=0)
clusters = kmeans.fit_predict(digits_proj)
kmeans.cluster_centers_.shape'''

In [ ]:
#without TSNE
#Creating the KMeans model and predict classes (n_clusters =4 ; due to 4 classes)
kmeans = KMeans(n_clusters=4, random_state=0)
clusters = kmeans.fit_predict(hog_X)
kmeans.cluster_centers_.shape

In [ ]:
clusters= pd.DataFrame(clusters,columns = ['ClassId_pred'])
clusters

In [ ]:
hog.reset_index(drop=True, inplace=True)
clusters.reset_index(drop=True, inplace=True)
result = pd.concat([hog, clusters], axis=1)
result['ClassId_pred'] = result['ClassId_pred'].map({0: 1, 1: 2,2:3, 3:4})
#switch labels due to results from confusion matrix
#result['ClassId_pred'] = result['ClassId_pred'].map({1: 1, 2: 2, 3:4, 4:3})
result

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(result['ClassId'],result['ClassId_pred']))
# Print confusion matrix
sns.heatmap(metrics.confusion_matrix(result['ClassId'],result['ClassId_pred']), annot=True, cmap='YlGn');

In [ ]:
print(metrics.classification_report(result['ClassId'],result['ClassId_pred']))

In [ ]:
'''# Create a labels array to match the learned cluster lables with the true labels
labels = np.zeros_like(clusters)
for i in range(4):
    mask = (clusters == i)
    labels[mask] = mode(hog.ClassId[mask])[0]'''

### HOG without TSNE        
        precision    recall  f1-score   support

           1       0.12      0.18      0.15       769
           2       0.06      0.70      0.10       195
           3       0.81      0.27      0.41      4759
           4       0.01      0.01      0.01       516

    accuracy                           0.25      6239
   macro avg       0.25      0.29      0.17      6239
weighted avg       0.63      0.25      0.33      6239

   ### HOG with TSNE        
        precision    recall  f1-score   support

           1       0.12      0.36      0.18       769
           2       0.00      0.04      0.01       195
           3       0.77      0.18      0.29      4759
           4       0.02      0.03      0.02       516

    accuracy                           0.18      6239
   macro avg       0.23      0.15      0.12      6239
weighted avg       0.61      0.18      0.24      6239

## Using HOG Image in Clustering

In [ ]:
image = io.imread("data/train_images/000f6bf48.jpg")

In [ ]:
from skimage.transform import resize
resized_img = resize(image, (64,128))

In [ ]:
plt.imshow(resized_img)
plt.xlim(0,128)
plt.ylim(0,64)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

In [ ]:
from skimage.feature import hog
fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True, channel_axis=-1)

In [ ]:
plt.imshow(hog_image)
plt.xlim(0,128)
plt.ylim(0,64)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

In [ ]:
pic_n = hog_image.reshape(hog_image.shape[0]*hog_image.shape[1],1)
kmeans = KMeans(n_clusters=2, random_state=0).fit(pic_n)
#clusters = kmeans.fit_predict(hog_image)
#kmeans.cluster_centers_.shape

In [ ]:
pic2show = kmeans.cluster_centers_[kmeans.labels_]

In [ ]:
cluster_pic = pic2show.reshape(hog_image.shape[0], hog_image.shape[1])
plt.figure(figsize=(12,10))
plt.imshow(cluster_pic);

In [ ]:
cluster_pic

In [ ]:
hog_image.shape

In [ ]:
kmeans.labels_.shape

In [ ]:
# self-written scripts
import sys
sys.path.insert(0, 'Python_Scripts')

import mask_conversion

In [ ]:
mask_pic = kmeans.labels_.reshape(hog_image.shape[0], hog_image.shape[1])

In [ ]:
mask_pic.shape

In [ ]:
list(set(kmeans.labels_))

In [ ]:

DIMENSION = (64,128)
encoded_pixels = mask_conversion.encode_pixel(mask_pic, 3)
mask_conversion.decode_pixel(DIMENSION,encoded_pixels=encoded_pixels,class_id=1)

In [ ]:
# get unique values in kmeans labels
labels = list(set(kmeans.labels_))

for label in labels:
    DIMENSION = mask_pic.shape
    encoded_pixels=mask_conversion.encode_pixel(mask_pic, label)
    
    mask = mask_conversion.decode_pixel(DIMENSION,encoded_pixels=encoded_pixels,class_id=1)

    plt.figure(figsize=(25, 8))
        
    ax = plt.subplot(len(labels), 1, label + 1)
    #plt.imshow(img)
    title = f'Label: {label}'
    
    plt.title(title, fontsize=16);
    plt.axis("off")

    plt.imshow(mask)